In [1]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [2]:
#import cell
import pandas as pd
import numpy as np
import statistics
import csv
import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

In [3]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [4]:
#Gets the master unemployed table
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [5]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
unemploymentData = query_job.to_dataframe()
print(unemploymentData.shape)
pd.set_option('display.max_columns', None)
unemploymentData.head(3)

(16833, 25)


,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,edu_post_hs,edu_unknown,top_category_employer1,top_category_employer2,top_category_employer3,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,<NA>,0,31-33,51,62,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,<NA>,0,48-49,51,56,76,35,0,0,0,<NA>,<NA>,0,0,0
2,6a5609f385912113b6f1014b958ed748,20220326,13,18089,11500,"Census Tract 115, Lake County, Indiana",39,0,<NA>,<NA>,<NA>,0,56,N/A,N/A,<NA>,<NA>,0,0,0,39,0,0,0,0


In [6]:
#Gets each tracks mean and std dev
#unlist has the master unemployment list
#b becomes the filtered list
unList = unemploymentData.values.tolist()
b_set = set(tuple(x) for x in unList)
b = [ list(x) for x in b_set ]

In [7]:
uuid = []
#makes a list of the unique uuid
for x in b:
    if(uuid.count(x[0]) == 0):
        uuid.append(x[0])

In [8]:
#setup for extract        
values = []
export = []

In [9]:
#for each value make a list of each weeks claims
for y in uuid:
    temp = [y]
    for x in b:
        if (x[0] == y):
            temp.append(x[6])
    values.append(temp)

In [10]:
for x in values:
    name = x[0]
    mean = statistics.mean(x[1:])
    if (len(x) > 2):
        stdev = statistics.stdev(x[1:])
    else:
        print("short")
    export.append([name, mean, stdev])

In [11]:
#Everything above this is testing
#Make the answer csv
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [12]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
predictionData = query_job.to_dataframe()

In [13]:
answers = predictionData.values.tolist()
#Answers[names, week]
#export[names, mean, stddev]
i = 0
for x in answers:
    for y in export:
        if(y.count(x[0]) == 1):
            answers[i].insert(1, y[1])
    i = i + 1

In [14]:
fields = ['uu_id', 'total_claims', 'week_number']
with open('submission_prediction_output.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(answers)